In [3]:
# Project 3: LLM-Powered Physics Assistant for Accelerator Research
# An intelligent AI assistant that understands particle physics and accelerator science
# Author : Abhipsa Acharya

import openai
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import re
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# For demonstration, we'll use a mock LLM interface
# In practice, you'd use OpenAI API, Hugging Face transformers, or local models

class MockLLMInterface:
    """
    Mock LLM interface for demonstration
    In practice, replace with actual LLM API calls
    """
    
    def __init__(self):
        self.model_name = "physics-assistant-7b"
        self.physics_knowledge = self._load_physics_knowledge()
    
    def _load_physics_knowledge(self):
        """Load physics knowledge base"""
        return {
            'accelerator_types': ['linear', 'circular', 'synchrotron', 'cyclotron'],
            'beam_parameters': ['energy', 'current', 'emittance', 'beta_function'],
            'collective_effects': ['space_charge', 'wake_fields', 'beam_beam'],
            'diagnostics': ['BPM', 'wire_scanner', 'synchrotron_light'],
            'optimization_targets': ['luminosity', 'beam_lifetime', 'emittance']
        }
    
    def generate_response(self, prompt: str, context: str = "") -> str:
        """
        Mock LLM response generation
        In practice: return openai.Completion.create() or similar
        """
        
        # Simple pattern matching for demonstration
        if "beam energy" in prompt.lower():
            return "Beam energy is a fundamental parameter that determines particle velocity and magnetic rigidity. For relativistic particles, E = γmc² where γ is the Lorentz factor."
        
        elif "space charge" in prompt.lower():
            return "Space charge effects arise from Coulomb repulsion between particles in the beam. They cause tune shifts proportional to beam current and inversely proportional to beam energy: Δν ∝ I/(γ³β³)."
        
        elif "optimization" in prompt.lower():
            return "Accelerator optimization typically involves multi-objective goals like maximizing luminosity while minimizing beam loss. Key parameters include β-functions, dispersion, and working point selection."
        
        elif "troubleshooting" in prompt.lower():
            return "Common accelerator issues include beam instabilities, orbit drift, and RF system problems. Systematic diagnosis starts with beam position monitors and loss detectors."
        
        else:
            return f"I understand you're asking about: {prompt[:100]}... Let me analyze this in the context of accelerator physics principles."

class PhysicsKnowledgeBase:
    """
    Structured physics knowledge for the LLM assistant
    """
    
    def __init__(self):
        self.accelerator_physics = {
            'beam_dynamics': {
                'linear_motion': 'Betatron oscillations around design orbit',
                'nonlinear_effects': 'Sextupole-induced amplitude dependent tune shifts',
                'synchrotron_motion': 'Longitudinal oscillations in RF bucket',
                'coupling': 'Cross-plane motion coupling through skew fields'
            },
            'collective_effects': {
                'space_charge': 'Coulomb forces between beam particles',
                'wake_fields': 'Electromagnetic fields left by leading particles',
                'beam_beam': 'Electromagnetic interaction between colliding beams',
                'electron_cloud': 'Secondary electron buildup in beam chamber'
            },
            'accelerator_systems': {
                'magnets': 'Dipoles for bending, quadrupoles for focusing',
                'rf_systems': 'Acceleration and longitudinal focusing',
                'vacuum': 'Ultra-high vacuum for beam lifetime',
                'diagnostics': 'Beam position, profile, and loss monitoring'
            }
        }
        
        self.formulas = {
            'relativistic_energy': 'E = γmc² where γ = 1/√(1-β²)',
            'betatron_tune': 'ν = (1/2π) ∮ (1/β(s)) ds',
            'space_charge_tune_shift': 'Δν = -r₀λ/(4πγ³β³ε)',
            'synchrotron_frequency': 'f_s = f₀√(eVh|η|/(2πγm))',
            'luminosity': 'L = f_c × N₁N₂/(4πσ_x σ_y) × H'
        }
        
        self.typical_values = {
            'electron_energy_lep': '100 GeV',
            'proton_energy_lhc': '6.5 TeV',
            'beam_current_synchrotron': '100-500 mA',
            'emittance_electron': '1-10 nm·rad',
            'beta_function': '1-100 m',
            'tune_electron': '10-50',
            'rf_frequency': '100-1000 MHz'
        }

class AcceleratorDataAnalyzer:
    """
    Analyzes accelerator data and provides physics insights
    """
    
    def __init__(self):
        self.analysis_functions = {
            'beam_loss': self._analyze_beam_loss,
            'orbit_stability': self._analyze_orbit,
            'tune_measurement': self._analyze_tune,
            'luminosity_evolution': self._analyze_luminosity,
            'instability_detection': self._detect_instabilities
        }
    
    def _analyze_beam_loss(self, data: Dict) -> Dict:
        """Analyze beam loss patterns"""
        if 'loss_monitors' not in data:
            return {'error': 'No loss monitor data provided'}
        
        loss_data = np.array(data['loss_monitors'])
        
        analysis = {
            'total_loss_rate': np.sum(loss_data),
            'peak_loss_location': np.argmax(loss_data),
            'loss_pattern': 'localized' if np.max(loss_data) > 3*np.mean(loss_data) else 'distributed',
            'recommendations': []
        }
        
        if analysis['total_loss_rate'] > 1000:  # Arbitrary threshold
            analysis['recommendations'].append('High beam loss detected - check collimation system')
        
        if analysis['loss_pattern'] == 'localized':
            analysis['recommendations'].append(f'Localized loss at position {analysis["peak_loss_location"]} - inspect local aperture')
        
        return analysis
    
    def _analyze_orbit(self, data: Dict) -> Dict:
        """Analyze beam orbit stability"""
        if 'bpm_readings' not in data:
            return {'error': 'No BPM data provided'}
        
        bpm_data = np.array(data['bpm_readings'])
        
        # Calculate orbit RMS and drift
        orbit_rms = np.std(bpm_data, axis=0) if len(bpm_data.shape) > 1 else np.std(bpm_data)
        orbit_drift = np.mean(np.diff(bpm_data, axis=0)) if len(bpm_data.shape) > 1 else np.mean(np.diff(bpm_data))
        
        analysis = {
            'orbit_rms_x': float(orbit_rms[0]) if len(bpm_data.shape) > 1 else float(orbit_rms),
            'orbit_rms_y': float(orbit_rms[1]) if len(bpm_data.shape) > 1 and bpm_data.shape[1] > 1 else 0.0,
            'orbit_drift': float(orbit_drift),
            'stability_rating': 'good' if orbit_rms.max() < 0.1 else 'poor',
            'recommendations': []
        }
        
        if analysis['orbit_rms_x'] > 0.5:
            analysis['recommendations'].append('Large horizontal orbit excursions - check steering magnets')
        
        if abs(analysis['orbit_drift']) > 0.01:
            analysis['recommendations'].append('Significant orbit drift detected - thermal effects or magnet stability issue')
        
        return analysis
    
    def _analyze_tune(self, data: Dict) -> Dict:
        """Analyze betatron tune measurements"""
        if 'tune_x' not in data or 'tune_y' not in data:
            return {'error': 'Tune data (tune_x, tune_y) required'}
        
        tune_x, tune_y = data['tune_x'], data['tune_y']
        
        # Check resonance proximity
        resonances = self._check_resonances(tune_x, tune_y)
        
        analysis = {
            'tune_x': tune_x,
            'tune_y': tune_y,
            'tune_separation': abs(tune_x - tune_y),
            'resonance_warnings': resonances,
            'working_point_quality': 'good' if len(resonances) == 0 else 'poor',
            'recommendations': []
        }
        
        if analysis['tune_separation'] < 0.01:
            analysis['recommendations'].append('Tunes too close - risk of linear coupling resonance')
        
        if len(resonances) > 0:
            analysis['recommendations'].append(f'Avoid resonances: {", ".join(resonances)}')
        
        return analysis
    
    def _check_resonances(self, tune_x: float, tune_y: float) -> List[str]:
        """Check proximity to dangerous resonances"""
        resonances = []
        
        # Integer resonance
        if abs(tune_x - round(tune_x)) < 0.05 or abs(tune_y - round(tune_y)) < 0.05:
            resonances.append('integer')
        
        # Half-integer resonance
        if abs(tune_x - round(tune_x) + 0.5) < 0.05 or abs(tune_y - round(tune_y) + 0.5) < 0.05:
            resonances.append('half-integer')
        
        # Sum resonance
        if abs((tune_x + tune_y) - round(tune_x + tune_y)) < 0.05:
            resonances.append('sum')
        
        # Difference resonance
        if abs((tune_x - tune_y) - round(tune_x - tune_y)) < 0.05:
            resonances.append('difference')
        
        return resonances
    
    def _analyze_luminosity(self, data: Dict) -> Dict:
        """Analyze luminosity evolution and optimization"""
        if 'luminosity_history' not in data:
            return {'error': 'Luminosity history data required'}
        
        lumi_data = np.array(data['luminosity_history'])
        time_points = np.arange(len(lumi_data))
        
        # Fit exponential decay: L(t) = L₀ * exp(-t/τ)
        if len(lumi_data) > 5:
            log_lumi = np.log(lumi_data + 1e-10)  # Avoid log(0)
            slope, intercept = np.polyfit(time_points, log_lumi, 1)
            lifetime = -1/slope if slope < 0 else float('inf')
        else:
            lifetime = float('inf')
        
        analysis = {
            'initial_luminosity': float(lumi_data[0]),
            'current_luminosity': float(lumi_data[-1]),
            'peak_luminosity': float(np.max(lumi_data)),
            'luminosity_lifetime': lifetime,
            'decay_rate': float(slope) if 'slope' in locals() else 0.0,
            'recommendations': []
        }
        
        if lifetime < 10:  # hours
            analysis['recommendations'].append('Short luminosity lifetime - check beam-beam effects and background')
        
        if analysis['current_luminosity'] < 0.5 * analysis['peak_luminosity']:
            analysis['recommendations'].append('Significant luminosity degradation - consider beam refresh')
        
        return analysis
    
    def _detect_instabilities(self, data: Dict) -> Dict:
        """Detect beam instabilities from various diagnostics"""
        instabilities = []
        
        # Check for coherent oscillations in BPM data
        if 'bpm_readings' in data:
            bpm_data = np.array(data['bpm_readings'])
            if len(bpm_data) > 10:
                # Simple FFT analysis for coherent modes
                fft_data = np.fft.fft(bpm_data.flatten())
                dominant_freq = np.argmax(np.abs(fft_data))
                if np.abs(fft_data[dominant_freq]) > 0.1 * len(bpm_data):
                    instabilities.append('coherent_oscillation')
        
        # Check tune spread for incoherent instabilities
        if 'tune_spread' in data and data['tune_spread'] > 0.01:
            instabilities.append('incoherent_tune_spread')
        
        # Check for sudden beam loss increases
        if 'loss_monitors' in data:
            loss_data = np.array(data['loss_monitors'])
            if np.any(np.diff(loss_data) > 5 * np.std(loss_data)):
                instabilities.append('sudden_beam_loss')
        
        return {
            'detected_instabilities': instabilities,
            'stability_status': 'stable' if len(instabilities) == 0 else 'unstable',
            'recommendations': self._get_instability_recommendations(instabilities)
        }
    
    def _get_instability_recommendations(self, instabilities: List[str]) -> List[str]:
        """Get recommendations for detected instabilities"""
        recommendations = []
        
        if 'coherent_oscillation' in instabilities:
            recommendations.append('Coherent instability detected - check feedback systems and impedance sources')
        
        if 'incoherent_tune_spread' in instabilities:
            recommendations.append('Large tune spread suggests space charge or nonlinear effects - reduce beam intensity')
        
        if 'sudden_beam_loss' in instabilities:
            recommendations.append('Sudden beam loss indicates possible equipment failure or beam-gas scattering')
        
        return recommendations

class LLMPhysicsAssistant:
    """
    Main LLM-powered physics assistant for accelerator research
    """
    
    def __init__(self, model_name: str = "physics-assistant-7b"):
        self.llm = MockLLMInterface()
        self.knowledge_base = PhysicsKnowledgeBase()
        self.data_analyzer = AcceleratorDataAnalyzer()
        self.conversation_history = []
        
        # System prompt for physics context
        self.system_prompt = """
        You are an expert accelerator physicist and AI assistant specializing in particle accelerator science. 
        You have deep knowledge of:
        - Beam dynamics and collective effects
        - Accelerator systems and diagnostics  
        - Data analysis and optimization
        - Troubleshooting and problem-solving
        
        Provide accurate, helpful responses that combine theoretical physics with practical experience.
        Always consider safety and beam protection when making recommendations.
        """
    
    def ask_physics_question(self, question: str, context_data: Optional[Dict] = None) -> Dict:
        """
        Main interface for asking physics questions
        """
        
        # Log the conversation
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # Analyze any provided data
        data_analysis = {}
        if context_data:
            data_analysis = self._analyze_context_data(context_data)
        
        # Prepare context for LLM
        context = self._prepare_context(question, data_analysis)
        
        # Generate LLM response
        llm_response = self.llm.generate_response(question, context)
        
        # Post-process and enhance response
        enhanced_response = self._enhance_response(llm_response, question, data_analysis)
        
        # Prepare final response
        response = {
            'timestamp': timestamp,
            'question': question,
            'answer': enhanced_response,
            'data_analysis': data_analysis,
            'confidence': self._estimate_confidence(question, enhanced_response),
            'follow_up_suggestions': self._generate_follow_ups(question),
            'relevant_formulas': self._extract_relevant_formulas(question),
            'references': self._get_references(question)
        }
        
        # Store in conversation history
        self.conversation_history.append(response)
        
        return response
    
    def _analyze_context_data(self, data: Dict) -> Dict:
        """Analyze provided accelerator data"""
        analysis_results = {}
        
        # Determine what type of analysis to perform based on data keys
        for analysis_type, analyzer_func in self.data_analyzer.analysis_functions.items():
            try:
                result = analyzer_func(data)
                if 'error' not in result:
                    analysis_results[analysis_type] = result
            except Exception as e:
                continue  # Skip analysis types that don't match the data
        
        return analysis_results
    
    def _prepare_context(self, question: str, data_analysis: Dict) -> str:
        """Prepare context string for LLM"""
        context_parts = [self.system_prompt]
        
        # Add relevant physics knowledge
        relevant_knowledge = self._extract_relevant_knowledge(question)
        if relevant_knowledge:
            context_parts.append(f"Relevant physics context: {relevant_knowledge}")
        
        # Add data analysis results (convert numpy types to native Python)
        if data_analysis:
            # Convert numpy types to JSON-serializable types
            serializable_analysis = self._convert_numpy_types(data_analysis)
            context_parts.append(f"Data analysis results: {json.dumps(serializable_analysis, indent=2)}")
        
        # Add conversation history context
        if len(self.conversation_history) > 0:
            recent_context = self.conversation_history[-3:]  # Last 3 exchanges
            context_parts.append(f"Recent conversation: {[h['question'] for h in recent_context]}")
        
        return "\n\n".join(context_parts)
    
    def _convert_numpy_types(self, obj):
        """Convert numpy types to native Python types for JSON serialization"""
        if isinstance(obj, dict):
            return {key: self._convert_numpy_types(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [self._convert_numpy_types(item) for item in obj]
        elif hasattr(obj, 'item'):  # numpy scalar
            return obj.item()
        elif hasattr(obj, 'tolist'):  # numpy array
            return obj.tolist()
        else:
            return obj
    
    def _extract_relevant_knowledge(self, question: str) -> str:
        """Extract relevant physics knowledge based on question keywords"""
        question_lower = question.lower()
        relevant_info = []
        
        # Check for relevant topics in knowledge base
        for category, items in self.knowledge_base.accelerator_physics.items():
            for topic, description in items.items():
                if topic.replace('_', ' ') in question_lower:
                    relevant_info.append(f"{topic}: {description}")
        
        # Add relevant formulas
        for formula_name, formula in self.knowledge_base.formulas.items():
            if any(keyword in question_lower for keyword in formula_name.split('_')):
                relevant_info.append(f"{formula_name}: {formula}")
        
        return "; ".join(relevant_info[:3])  # Limit to avoid overwhelming context
    
    def _enhance_response(self, llm_response: str, question: str, data_analysis: Dict) -> str:
        """Enhance LLM response with data analysis insights"""
        enhanced_parts = [llm_response]
        
        # Add data-driven insights
        if data_analysis:
            enhanced_parts.append("\n🔍 **Data Analysis Insights:**")
            
            for analysis_type, results in data_analysis.items():
                if 'recommendations' in results and results['recommendations']:
                    enhanced_parts.append(f"\n**{analysis_type.replace('_', ' ').title()}:**")
                    for rec in results['recommendations']:
                        enhanced_parts.append(f"• {rec}")
        
        # Add safety warnings if relevant
        safety_warnings = self._check_safety_warnings(question, data_analysis)
        if safety_warnings:
            enhanced_parts.append(f"\n⚠️ **Safety Considerations:** {safety_warnings}")
        
        return "".join(enhanced_parts)
    
    def _check_safety_warnings(self, question: str, data_analysis: Dict) -> str:
        """Check for safety-related warnings"""
        warnings = []
        
        if "high current" in question.lower() or "high intensity" in question.lower():
            warnings.append("High intensity beams require careful beam loss monitoring and machine protection systems.")
        
        if any("beam_loss" in analysis and analysis.get("total_loss_rate", 0) > 1000 
               for analysis in data_analysis.values() if isinstance(analysis, dict)):
            warnings.append("High beam loss detected - ensure beam dump systems are operational.")
        
        if any("instability" in str(analysis) for analysis in data_analysis.values()):
            warnings.append("Beam instabilities can cause equipment damage - monitor beam abort thresholds.")
        
        return " ".join(warnings)
    
    def _estimate_confidence(self, question: str, response: str) -> float:
        """Estimate confidence in the response"""
        # Simple heuristic based on question complexity and response detail
        confidence = 0.8  # Base confidence
        
        # Reduce confidence for very complex or vague questions
        if len(question.split()) > 20:
            confidence -= 0.1
        
        if any(word in question.lower() for word in ['complex', 'complicated', 'unsure', 'maybe']):
            confidence -= 0.2
        
        # Increase confidence for specific technical questions
        if any(term in question.lower() for term in ['formula', 'equation', 'calculate', 'measure']):
            confidence += 0.1
        
        # Increase confidence if response includes specific numbers or formulas
        if re.search(r'\d+', response) or '=' in response:
            confidence += 0.1
        
        return max(0.1, min(1.0, confidence))
    
    def _generate_follow_ups(self, question: str) -> List[str]:
        """Generate relevant follow-up questions"""
        follow_ups = []
        
        if "beam loss" in question.lower():
            follow_ups.extend([
                "What are the typical beam loss thresholds for machine protection?",
                "How can we optimize the collimation system to reduce losses?",
                "What diagnostics are best for localizing beam loss sources?"
            ])
        
        elif "tune" in question.lower():
            follow_ups.extend([
                "How do space charge effects modify the working point?",
                "What is the optimal tune separation to avoid coupling?",
                "How can we measure and correct tune variations?"
            ])
        
        elif "luminosity" in question.lower():
            follow_ups.extend([
                "What factors limit the luminosity lifetime?",
                "How can beam-beam effects be minimized?",
                "What is the optimal β-function at the interaction point?"
            ])
        
        else:
            follow_ups.extend([
                "What are the key parameters to monitor for this system?",
                "How can we optimize performance while maintaining stability?",
                "What are the main physics limitations?"
            ])
        
        return follow_ups[:3]  # Return top 3 most relevant
    
    def _extract_relevant_formulas(self, question: str) -> List[str]:
        """Extract formulas relevant to the question"""
        relevant_formulas = []
        
        for formula_name, formula in self.knowledge_base.formulas.items():
            # Check if any keywords from formula name appear in question
            keywords = formula_name.split('_')
            if any(keyword in question.lower() for keyword in keywords):
                relevant_formulas.append(f"{formula_name.replace('_', ' ').title()}: {formula}")
        
        return relevant_formulas
    
    def _get_references(self, question: str) -> List[str]:
        """Get relevant references based on question topic"""
        references = []
        
        if any(term in question.lower() for term in ['beam dynamics', 'optics', 'lattice']):
            references.append("Wiedemann, H. 'Particle Accelerator Physics' (2007)")
            references.append("Chao, A. 'Handbook of Accelerator Physics' (2013)")
        
        if any(term in question.lower() for term in ['collective effects', 'wake', 'instability']):
            references.append("Ng, K.Y. 'Physics of Intensity Dependent Beam Instabilities' (2006)")
            references.append("Chao, A. 'Physics of Collective Beam Instabilities' (1993)")
        
        if any(term in question.lower() for term in ['rf', 'cavity', 'acceleration']):
            references.append("Padamsee, H. 'RF Superconductivity for Accelerators' (2008)")
        
        return references[:3]  # Limit to most relevant
    
    def get_optimization_suggestions(self, target: str, current_data: Dict) -> Dict:
        """Provide optimization suggestions for specific targets"""
        
        optimization_strategies = {
            'luminosity': {
                'parameters': ['beta_functions', 'beam_current', 'emittance'],
                'strategies': [
                    'Reduce β* at interaction point to decrease beam size',
                    'Increase beam current while managing beam-beam effects',
                    'Minimize emittance through improved injection and damping'
                ],
                'constraints': ['Beam-beam tune shift', 'Dynamic aperture', 'Detector backgrounds']
            },
            'beam_lifetime': {
                'parameters': ['vacuum_pressure', 'orbit_stability', 'tune_working_point'],
                'strategies': [
                    'Improve vacuum system to reduce beam-gas scattering',
                    'Stabilize orbit to minimize aperture limitations',
                    'Optimize working point to avoid resonances'
                ],
                'constraints': ['Vacuum conductance', 'Magnet stability', 'Power supply ripple']
            },
            'emittance': {
                'parameters': ['damping_partition', 'rf_voltage', 'insertion_devices'],
                'strategies': [
                    'Optimize damping partition numbers for minimum emittance',
                    'Adjust RF voltage for optimal synchrotron radiation damping',
                    'Minimize dispersion in insertion device regions'
                ],
                'constraints': ['Synchrotron radiation power', 'Quantum fluctuations', 'Lattice constraints']
            }
        }
        
        if target.lower() not in optimization_strategies:
            return {'error': f'Optimization target "{target}" not recognized'}
        
        strategy = optimization_strategies[target.lower()]
        
        # Analyze current data for specific recommendations
        data_analysis = self._analyze_context_data(current_data)
        
        specific_recommendations = []
        if 'tune_measurement' in data_analysis:
            tune_data = data_analysis['tune_measurement']
            if len(tune_data.get('resonance_warnings', [])) > 0:
                specific_recommendations.append('Move working point away from resonances')
        
        if 'beam_loss' in data_analysis:
            loss_data = data_analysis['beam_loss']
            if loss_data.get('total_loss_rate', 0) > 100:
                specific_recommendations.append('Address beam losses before optimization')
        
        return {
            'target': target,
            'key_parameters': strategy['parameters'],
            'general_strategies': strategy['strategies'],
            'constraints': strategy['constraints'],
            'specific_recommendations': specific_recommendations,
            'current_status': data_analysis
        }

def demonstrate_physics_assistant():
    """Demonstrate the LLM Physics Assistant with example interactions"""
    
    print("🤖 LLM-Powered Physics Assistant for Accelerator Research")
    print("=" * 60)
    
    # Initialize the assistant
    assistant = LLMPhysicsAssistant()
    
    # Example 1: Basic physics question
    print("\n📚 Example 1: Basic Physics Question")
    print("-" * 40)
    
    response1 = assistant.ask_physics_question(
        "What causes space charge effects in particle beams and how do they scale with beam parameters?"
    )
    
    print(f"Question: {response1['question']}")
    print(f"Answer: {response1['answer']}")
    print(f"Confidence: {response1['confidence']:.2f}")
    print(f"Relevant formulas: {response1['relevant_formulas']}")
    
    # Example 2: Data analysis question
    print("\n📊 Example 2: Data Analysis with Accelerator Data")
    print("-" * 40)
    
    # Simulate accelerator data
    sample_data = {
        'bpm_readings': np.random.normal(0, 0.1, (100, 2)),  # BPM data with some noise
        'loss_monitors': np.random.exponential(10, 50),      # Loss monitor readings
        'tune_x': 10.15,                                     # Horizontal tune
        'tune_y': 5.22,                                      # Vertical tune
        'luminosity_history': 100 * np.exp(-np.linspace(0, 2, 50))  # Decaying luminosity
    }
    
    response2 = assistant.ask_physics_question(
        "Can you analyze the beam stability and performance from this data?",
        context_data=sample_data
    )
    
    print(f"Question: {response2['question']}")
    print(f"Answer: {response2['answer']}")
    print(f"Data Analysis Summary:")
    for analysis_type, results in response2['data_analysis'].items():
        print(f"  • {analysis_type}: {results.get('recommendations', ['No issues'])}")
    
    # Example 3: Optimization request
    print("\n🎯 Example 3: Optimization Suggestions")
    print("-" * 40)
    
    optimization = assistant.get_optimization_suggestions(
        target='luminosity',
        current_data=sample_data
    )
    
    print(f"Optimization Target: {optimization['target']}")
    print(f"Key Parameters: {', '.join(optimization['key_parameters'])}")
    print("Strategies:")
    for strategy in optimization['general_strategies']:
        print(f"  • {strategy}")
    
    # Example 4: Follow-up conversation
    print("\n💬 Example 4: Follow-up Questions")
    print("-" * 40)
    
    follow_up_question = "How can we measure tune shifts experimentally?"
    response4 = assistant.ask_physics_question(follow_up_question)
    
    print(f"Follow-up: {follow_up_question}")
    print(f"Answer: {response4['answer']}")
    print("Suggested next questions:")
    for suggestion in response4['follow_up_suggestions']:
        print(f"  • {suggestion}")
    
    # Example 5: Troubleshooting scenario
    print("\n🔧 Example 5: Troubleshooting Assistance")
    print("-" * 40)
    
    # Simulate problematic data
    problem_data = {
        'bpm_readings': np.random.normal(0, 0.5, (100, 2)),  # Large orbit excursions
        'loss_monitors': np.random.exponential(100, 50),     # High losses
        'tune_x': 10.5,                                      # Near half-integer
        'tune_y': 5.5                                        # Near half-integer
    }
    
    response5 = assistant.ask_physics_question(
        "We're experiencing high beam losses and orbit instabilities. What could be the causes and solutions?",
        context_data=problem_data
    )
    
    print(f"Problem: {response5['question']}")
    print(f"Diagnosis: {response5['answer']}")
    
    # Summary of assistant capabilities
    print("\n🎉 Assistant Capabilities Summary")
    print("-" * 40)
    print("✅ Physics Q&A with contextual understanding")
    print("✅ Real-time accelerator data analysis")
    print("✅ Optimization strategy recommendations")
    print("✅ Troubleshooting and problem diagnosis")
    print("✅ Safety warnings and considerations")
    print("✅ Follow-up question suggestions")
    print("✅ Relevant formula and reference retrieval")
    
    print(f"\n📈 Conversation History: {len(assistant.conversation_history)} interactions")
    
    return assistant

class AcceleratorChatInterface:
    """
    Simple chat interface for the physics assistant
    """
    
    def __init__(self):
        self.assistant = LLMPhysicsAssistant()
        self.running = True
        
    def start_chat(self):
        """Start interactive chat session"""
        
        print("🤖 Accelerator Physics Assistant")
        print("Type 'help' for commands, 'quit' to exit")
        print("-" * 50)
        
        while self.running:
            try:
                user_input = input("\n💬 You: ").strip()
                
                if user_input.lower() in ['quit', 'exit', 'bye']:
                    print("👋 Goodbye! Keep those beams stable!")
                    self.running = False
                    continue
                
                elif user_input.lower() == 'help':
                    self.show_help()
                    continue
                
                elif user_input.lower() == 'history':
                    self.show_history()
                    continue
                
                elif user_input.lower().startswith('analyze'):
                    self.handle_data_analysis()
                    continue
                
                elif user_input.lower().startswith('optimize'):
                    self.handle_optimization()
                    continue
                
                elif not user_input:
                    continue
                
                # Process regular question
                response = self.assistant.ask_physics_question(user_input)
                
                print(f"\n🤖 Assistant: {response['answer']}")
                
                if response['relevant_formulas']:
                    print(f"\n📐 Relevant formulas:")
                    for formula in response['relevant_formulas']:
                        print(f"   • {formula}")
                
                if response['confidence'] < 0.6:
                    print(f"\n⚠️ Note: Low confidence ({response['confidence']:.2f}) - consider consulting additional sources")
                
                if response['follow_up_suggestions']:
                    print(f"\n💡 You might also ask:")
                    for suggestion in response['follow_up_suggestions'][:2]:
                        print(f"   • {suggestion}")
            
            except KeyboardInterrupt:
                print("\n👋 Goodbye!")
                self.running = False
            except Exception as e:
                print(f"\n❌ Error: {str(e)}")
    
    def show_help(self):
        """Show help information"""
        help_text = """
🆘 Available Commands:
• Just type your physics question naturally
• 'analyze' - Interactive data analysis
• 'optimize <target>' - Get optimization suggestions
• 'history' - Show conversation history
• 'help' - Show this help
• 'quit' - Exit the assistant

💡 Example questions:
• "What causes tune shifts in electron storage rings?"
• "How do I optimize luminosity in a collider?"
• "What are the main sources of beam instabilities?"
• "Explain space charge effects and their mitigation"
"""
        print(help_text)
    
    def show_history(self):
        """Show conversation history"""
        if not self.assistant.conversation_history:
            print("📝 No conversation history yet.")
            return
        
        print("📝 Recent conversation history:")
        for i, interaction in enumerate(self.assistant.conversation_history[-5:], 1):
            print(f"{i}. Q: {interaction['question'][:60]}...")
            print(f"   A: {interaction['answer'][:80]}...")
            print()
    
    def handle_data_analysis(self):
        """Handle interactive data analysis"""
        print("📊 Data Analysis Mode")
        print("Please provide data in JSON format or use sample data")
        
        use_sample = input("Use sample data? (y/n): ").lower().startswith('y')
        
        if use_sample:
            sample_data = {
                'bpm_readings': np.random.normal(0, 0.2, (50, 2)).tolist(),
                'loss_monitors': np.random.exponential(20, 25).tolist(),
                'tune_x': 10.3,
                'tune_y': 5.8,
                'luminosity_history': (100 * np.exp(-np.linspace(0, 1, 30))).tolist()
            }
            
            response = self.assistant.ask_physics_question(
                "Please analyze this accelerator data for any issues or trends",
                context_data=sample_data
            )
            
            print(f"\n🔍 Analysis: {response['answer']}")
        else:
            print("Data input functionality would be implemented here...")
    
    def handle_optimization(self):
        """Handle optimization requests"""
        print("🎯 Optimization Assistant")
        
        targets = ['luminosity', 'beam_lifetime', 'emittance']
        print("Available targets:", ", ".join(targets))
        
        target = input("Optimization target: ").strip().lower()
        
        if target in targets:
            sample_data = {'bpm_readings': [[0.1, 0.05]], 'tune_x': 10.2, 'tune_y': 5.7}
            
            optimization = self.assistant.get_optimization_suggestions(target, sample_data)
            
            print(f"\n🎯 Optimizing: {optimization['target']}")
            print("📋 Strategies:")
            for strategy in optimization['general_strategies']:
                print(f"   • {strategy}")
        else:
            print("❌ Target not recognized. Please choose from available options.")

class PhysicsCalculator:
    """
    Physics calculator for common accelerator formulas
    """
    
    def __init__(self):
        self.constants = {
            'c': 299792458,           # Speed of light [m/s]
            'e': 1.602176634e-19,     # Elementary charge [C]
            'm_e': 9.1093837015e-31,  # Electron mass [kg]
            'm_p': 1.67262192369e-27, # Proton mass [kg]
            'r_e': 2.8179403262e-15,  # Classical electron radius [m]
            'epsilon_0': 8.8541878128e-12  # Vacuum permittivity [F/m]
        }
    
    def relativistic_gamma(self, energy_gev: float, particle: str = 'electron') -> float:
        """Calculate relativistic gamma factor"""
        if particle.lower() == 'electron':
            rest_energy = 0.000511  # GeV
        elif particle.lower() == 'proton':
            rest_energy = 0.938272  # GeV
        else:
            raise ValueError("Particle must be 'electron' or 'proton'")
        
        return energy_gev / rest_energy
    
    def space_charge_tune_shift(self, beam_current_ma: float, energy_gev: float, 
                               emittance_nm: float, beta_m: float) -> float:
        """Calculate space charge tune shift"""
        gamma = self.relativistic_gamma(energy_gev)
        beta = np.sqrt(1 - 1/gamma**2)
        
        # Simplified formula (actual calculation more complex)
        r_0 = self.constants['r_e']
        I = beam_current_ma * 1e-3  # Convert to A
        lambda_beam = I / (self.constants['e'] * self.constants['c'])
        epsilon = emittance_nm * 1e-9
        
        tune_shift = -(r_0 * lambda_beam * beta_m) / (4 * np.pi * gamma**3 * beta**3 * epsilon)
        
        return tune_shift
    
    def synchrotron_frequency(self, rf_voltage_mv: float, harmonic: int, 
                             energy_gev: float, momentum_compaction: float) -> float:
        """Calculate synchrotron oscillation frequency"""
        gamma = self.relativistic_gamma(energy_gev)
        beta = np.sqrt(1 - 1/gamma**2)
        
        eta = momentum_compaction - 1/gamma**2  # Phase slip factor
        
        # Synchrotron frequency
        V_rf = rf_voltage_mv * 1e6  # Convert to V
        omega_rf = 2 * np.pi * harmonic * self.constants['c'] / (2 * np.pi * 100)  # Simplified
        
        omega_s = np.sqrt(self.constants['e'] * V_rf * harmonic * abs(eta) / 
                         (2 * np.pi * gamma * 0.511e6 / self.constants['c']**2))
        
        return omega_s / (2 * np.pi)  # Convert to Hz

def main():
    """Main execution function"""
    
    print("🚀 Starting Project 3: LLM-Powered Physics Assistant")
    print("=" * 60)
    
    # Choose demonstration mode
    print("\nSelect mode:")
    print("1. Automated demonstration (shows capabilities)")
    print("2. Interactive chat (ask questions)")
    print("3. Physics calculator (formula calculations)")
    
    try:
        choice = input("\nEnter choice (1-3): ").strip()
        
        if choice == '1':
            print("\n🤖 Running automated demonstration...")
            assistant = demonstrate_physics_assistant()
            
        elif choice == '2':
            print("\n💬 Starting interactive chat...")
            chat = AcceleratorChatInterface()
            chat.start_chat()
            
        elif choice == '3':
            print("\n🧮 Physics Calculator Mode")
            calc = PhysicsCalculator()
            
            # Example calculations
            print("\nExample calculations:")
            
            energy = 3.0  # GeV
            gamma = calc.relativistic_gamma(energy, 'electron')
            print(f"Relativistic γ for {energy} GeV electron: {gamma:.2f}")
            
            tune_shift = calc.space_charge_tune_shift(
                beam_current_ma=100, energy_gev=energy, 
                emittance_nm=10, beta_m=10
            )
            print(f"Space charge tune shift: {tune_shift:.6f}")
            
            synch_freq = calc.synchrotron_frequency(
                rf_voltage_mv=3.0, harmonic=400, 
                energy_gev=energy, momentum_compaction=0.01
            )
            print(f"Synchrotron frequency: {synch_freq:.3f} Hz")
            
        else:
            print("Invalid choice. Running demonstration...")
            demonstrate_physics_assistant()
    
    except KeyboardInterrupt:
        print("\n\n👋 Goodbye!")
    
    print("\n✅ Project 3 Complete!")
    print("The LLM Physics Assistant demonstrates:")
    print("• Natural language physics Q&A")
    print("• Real-time data analysis and insights")
    print("• Optimization recommendations")
    print("• Troubleshooting assistance")
    print("• Safety warnings and best practices")
    print("• Interactive chat capabilities")

if __name__ == "__main__":
    main()

🚀 Starting Project 3: LLM-Powered Physics Assistant

Select mode:
1. Automated demonstration (shows capabilities)
2. Interactive chat (ask questions)
3. Physics calculator (formula calculations)

Enter choice (1-3): 1

🤖 Running automated demonstration...
🤖 LLM-Powered Physics Assistant for Accelerator Research

📚 Example 1: Basic Physics Question
----------------------------------------
Question: What causes space charge effects in particle beams and how do they scale with beam parameters?
Answer: Space charge effects arise from Coulomb repulsion between particles in the beam. They cause tune shifts proportional to beam current and inversely proportional to beam energy: Δν ∝ I/(γ³β³).
Confidence: 0.80
Relevant formulas: ['Space Charge Tune Shift: Δν = -r₀λ/(4πγ³β³ε)']

📊 Example 2: Data Analysis with Accelerator Data
----------------------------------------
Question: Can you analyze the beam stability and performance from this data?
Answer: I understand you're asking about: Can you an